In [1]:
# Import required libraries and install any necessary packages
import spacy
import warnings

from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [2]:
# reading the data in doccano jsonl output format
with open('data/admin.jsonl', 'r') as f:
    lines = list(f)

training_data: list = []

for line in lines:
    row = json.loads(line)
    if row['label']:
        training_data.append(  [ row["text"], { "entities": row["label"] } ] )

print(len(training_data))

79


In [3]:
training_data[0]

['"#1 fruit forward lots | 1 . 2 ell\'sworth black honey orange , berry fruit characteristics , green grapes & lemon crisp acidity . suitable for all brew methods . origin chikmagalur , karnataka process hand picked cherries are pulped , sun dried on raised beds and then eventually washed before they are hulled . dry aroma mild florals wet aroma red ripe fruits based on sensory evaluation orange , berry fruit characteristics , green grapes & lemon crisp acidity . med body , clean mouthfeel & long prevailing aftertaste . varietal chandragiri roast profile light-med roast with variable drum speed and a lower charge temperature to achieve a 13% dtr . altitude 1130 masl minimum resting period filter 6 days | espresso 14 days a song that pairs well roaster thoughts : as black honey produces fruit forward flavors and consists of jammy characteristics . we roasted this coffee at 205 degrees to extract the dense properties , fruitier flavor notes and a heavier body . after cupping the coffee a

In [4]:
# Define a function to create spaCy DocBin objects from the annotated data
def get_spacy_doc(file, data):
  # Create a blank spaCy pipeline
  nlp = spacy.blank('en')
  db = DocBin()

  # Iterate through the data
  for text, annot in tqdm(data):
    doc = nlp.make_doc(text)
    annot = annot['entities']

    ents = []
    entity_indices = []

    # Extract entities from the annotations
    for start, end, label in annot:
      skip_entity = False
      for idx in range(start, end):
        if idx in entity_indices:
          skip_entity = True
          break
      if skip_entity:
        continue

      entity_indices = entity_indices + list(range(start, end))
      try:
        span = doc.char_span(start, end, label=label, alignment_mode='strict')
      except:
        continue

      if span is None:
        # Log errors for annotations that couldn't be processed
        err_data = str([start, end]) + "    " + str(text) + "\n"
        file.write(err_data)
      else:
        ents.append(span)

    try:
      doc.ents = ents
      db.add(doc)
    except:
      pass

  return db

In [5]:
# Split the annotated data into training and testing sets
from sklearn.model_selection import train_test_split
train, test = train_test_split(training_data, test_size=0.1)

# Display the number of items in the training and testing sets
print(len(train), len(test))

# Open a file to log errors during annotation processing
file = open('train_file_try_1.txt','w')

# Create spaCy DocBin objects for training and testing data
db = get_spacy_doc(file, train)
db.to_disk('train_doccano_try_1.spacy')

db = get_spacy_doc(file, test)
db.to_disk('test_doccano_try_1.spacy')

# Close the error log file
file.close()

71 8


/home/snehil/.dev_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/snehil/.dev_env/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/snehil/.dev_env/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
100%|██████████| 8/8 [00:00<00:00, 664.77it/s]


In [6]:
# configure spacy for custom NER model , using a base config file - https://spacy.io/usage/training#config

!python -m spacy init fill-config base_config.cfg config.cfg

/home/snehil/.dev_env/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/snehil/.dev_env/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


### Model Training

In [7]:
!python -m spacy train config.cfg  --output  output_try_1  --paths.train train_doccano_try_1.spacy  --paths.dev  test_doccano_try_1.spacy --gpu-id 0

/home/snehil/.dev_env/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
ℹ Saving to output directory: output_try_1
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
/home/snehil/.dev_env/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/snehil/.dev_env/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/snehil/.dev_env/lib/python3.12/site-pa

### Loading and trying the model with new data

In [14]:
# Import the spaCy library
import spacy

# Load the trained spaCy NER model from the specified path
nlp = spacy.load('output_try_1/model-best')


In [12]:
text = """
This is the first Indian coffee we are adding to our travellers addition.  These are exquisite Indian  coffee beans, meticulously cultivated in the lush highlands of Chikmagalur. Kerehaklu is  a green haven tucked away in the Western Ghats. The plantation is spread across 250 acres and is a 15-minute drive from the closest village of Aldur, making it a secluded getaway amid fascinating flora, fauna and fungi. This experimental batch is proccesed with Pichia yeast anoxic fermentation. 

Tasting Notes:

Kerehaklu coffee is a true sensory delight, offering a symphony of distinct tasting notes that will enchant your palate. At first sip, you'll encounter the gentle and toasty sweetness of caramel, followed by the refreshing strawbery notes. The flavor journey continues with the delightful vanilla. 

As the coffee dances across your taste buds, subtle hints of creamy vanilla emerge, adding a velvety richness to the overall profile.

The finishing note is a delicate and aromatic rose essence that leaves a floral aftertaste. 

 These remarkable coffee beans are a testament to the dedication and expertise of Pranoy & Ajoy  from the  Kerehaklu plantation family, who nurture them at each stage. The result is a truly unique and memorable coffee that invites you to savor each sip, allowing you to explore a world of flavors within the confines of your cup. Kerehaklu Pichia Yeast Anoxic fermented Coffee is the perfect companion for those seeking a refined and adventurous coffee experience.

Cultivated in the pristine regions of India and nurtured with care, these beans offer a unique flavor profile that reflects the terroir of its origin. With each sip, you'll taste the earthy undertones, hints of spice, and a velvety smoothness that defines Indian coffee.
"""

In [13]:
# Process the extracted text using the loaded spaCy NER model
doc = nlp(text)

# Iterate through the named entities (entities) recognized by the model
for ent in doc.ents:
  # Print the recognized text and its corresponding label
  print(ent.text, "  ->>>>  ", ent.label_)

Chikmagalur   ->>>>   LOCATION
Kerehaklu is   ->>>>   NAME
Aldur,   ->>>>   LOCATION
caramel   ->>>>   TASTING NOTES
vanilla   ->>>>   TASTING NOTES
rose   ->>>>   TASTING NOTES
Pranoy &   ->>>>   FARMER
Ajoy     ->>>>   FARMER
spice   ->>>>   TASTING NOTES


In [22]:
text ="""
Coffee Tasting Notes (Not flavours): Clean, Citrus, Pleasant, Short Aftertaste

Product is 100% Arabica Coffee direct from our farm. This variant is part of our mission to introduce consumers to pure Coffee.

(Cinnamon Roast) Light roasted coffee is the reward for hardwork by speciality Coffee roasters all around the world. This variant of Coffee which cannot be produced by low quality coffee beans. Light Roasted Coffee bean retains its origin flavours. Unique characteristics of Coffee is retained (which will be lost as roast level increases). Only A graded Arabica Coffee beans is used to make this roast.


"""

In [23]:
# Process the extracted text using the loaded spaCy NER model
doc = nlp(text)

# Iterate through the named entities (entities) recognized by the model
for ent in doc.ents:
  # Print the recognized text and its corresponding label
  print(ent.text, "  ->>>>  ", ent.label_)

Clean   ->>>>   TASTING NOTES
Citrus   ->>>>   TASTING NOTES
Pleasant   ->>>>   TASTING NOTES
Short   ->>>>   AFTERTASTE
Arabica   ->>>>   COFFEE TYPE
Light   ->>>>   ROAST LEVEL
